In [6]:
import numpy as np
import tensorflow as tf

In [7]:
tf.layers.Conv2D

tensorflow.python.layers.convolutional.Conv2D

In [35]:
# My Code
import numpy as np
import tensorflow as tf
from pandas_panel import *
# !pwd

batch_size = 12
seq_len = 12  # Number of periods Looking back in series.
n_channels = tt.table.values.shape[1]  # Number of variables in series collection.
channels = 1  # Grey scale.
forecast = 1
raw = tt.table.values
raw.shape[0]
tt.table.head()

,CPIAUCSL_PCH,UNRATE,A191RL1Q225SBEA
DATE,,,
1948-01-01,1.15335,3.4,6.0
1948-02-01,-0.04223,3.8,6.0
1948-03-01,-0.71821,4.0,6.0
1948-04-01,1.36170,3.9,6.7
1948-05-01,0.79765,3.5,6.7


In [40]:
break_point = (raw.shape[0] - (raw.shape[0] % seq_len))
x_data = raw[:break_point, :]
print(f"x data shape: {x_data.shape}")
y_data = raw[1: break_point+1, :]
print(f"Y data shape: {y_data.shape}")
x_train = x_data[:-seq_len, :]
x_test = x_data[-seq_len:, :]

x_train = x_data[:-seq_len, :]
y_train = x_train[:, 0]

x_batches = x_train.reshape(-1, seq_len, n_channels)
y_batches = y_train.reshape(-1, seq_len, 1)
print(x_batches.shape)
print(y_batches.shape)

y_test = y_data[-seq_len:, :]
y_test = y_test[:, 0]

x data shape: (840, 3)
Y data shape: (840, 3)
(69, 12, 3)
(69, 12, 1)


In [10]:
lstm_size = 9
lstm_layers = 2
batch_size = 24
seq_len = 48
learning_rate = 0.001
epochs = 1000

In [43]:
print(x_batches.shape)

(69, 12, 3)


In [80]:
tf.reset_default_graph()

In [81]:
# Setting up feeder
inputs_ = tf.placeholder(tf.float32,
                         [None, seq_len, n_channels],
                        name="input")

output_ = tf.placeholder(tf.float32,
                      [None, seq_len, 1],
                      name="output")

keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

conv1 = tf.layers.conv1d(inputs=inputs_,
                        filters=1,
                        kernel_size=2,
                        strides=1,
                        padding="same",
                        activation=tf.nn.relu)

In [82]:
conv1

<tf.Tensor 'conv1d/Relu:0' shape=(?, 12, 1) dtype=float32>

In [83]:
n_ch = 3
# Construct the LSTM inputs and LSTM cells
lstm_in = tf.transpose(conv1, [1, 0, 2])
lstm_in = tf.reshape(lstm_in, [-1, n_ch])

# To cells
lstm_in = tf.layers.dense(lstm_in, lstm_size, activation=None)

# Open up the tensor into a list of seq_len pieces
lstm_in = tf.split(lstm_in, seq_len, 0)

# Add LSTM layers
lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
drop = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=keep_prob_)
cell = tf.nn.rnn_cell.MultiRNNCell([drop] * lstm_layers)
initial_state = cell.zero_state(batch_size, tf.float32)

In [84]:
lstm_in

[<tf.Tensor 'split:0' shape=(?, 9) dtype=float32>,
 <tf.Tensor 'split:1' shape=(?, 9) dtype=float32>,
 <tf.Tensor 'split:2' shape=(?, 9) dtype=float32>,
 <tf.Tensor 'split:3' shape=(?, 9) dtype=float32>,
 <tf.Tensor 'split:4' shape=(?, 9) dtype=float32>,
 <tf.Tensor 'split:5' shape=(?, 9) dtype=float32>,
 <tf.Tensor 'split:6' shape=(?, 9) dtype=float32>,
 <tf.Tensor 'split:7' shape=(?, 9) dtype=float32>,
 <tf.Tensor 'split:8' shape=(?, 9) dtype=float32>,
 <tf.Tensor 'split:9' shape=(?, 9) dtype=float32>,
 <tf.Tensor 'split:10' shape=(?, 9) dtype=float32>,
 <tf.Tensor 'split:11' shape=(?, 9) dtype=float32>]

In [89]:
outputs, final_state = tf.nn.s(cell,
                                         lstm_in,
                                        dtype=tf.float32,
                                        initial_state=initial_state)

ValueError: Shape (9, ?) must have rank at least 3

In [86]:
loss = tf.sqrt(tf.reduce_mean(tf.sqrt(output_ - outputs)))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

In [87]:
outputs

[<tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/dropout/mul:0' shape=(12, 9) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1_1/dropout/mul:0' shape=(12, 9) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1_2/dropout/mul:0' shape=(12, 9) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1_3/dropout/mul:0' shape=(12, 9) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1_4/dropout/mul:0' shape=(12, 9) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1_5/dropout/mul:0' shape=(12, 9) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1_6/dropout/mul:0' shape=(12, 9) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1_7/dropout/mul:0' shape=(12, 9) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1_8/dropout/mul:0' shape=(12, 9) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1_9/dropout/mul:0' shape=(12, 9) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1_10/dropout/mul:0' shape=(12, 9) dtype=float32>,
 <tf.Tensor

In [88]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        for ep in range(epochs):
            sess.run(training_op,
                    feed_dict={
                        inputs_: x_batches,
                        outputs: y_batches
                    })

TypeError: unhashable type: 'list'

In [41]:
x_train.shape

(828, 3)